In [ ]:
# IRIS dataset test example

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR) 
import keras
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
# Load the Iris dataset from TensorFlow Datasets
(ds_train, ds_vaidation, ds_test), ds_info = tfds.load(
    'iris',
    split=['train[:60%]','train[60%:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Prepare the data: batch, shuffle, and prefetch
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS = 100

def preprocess(features, label):
    features = tf.cast(features, tf.float64)
    label = tf.cast(label, tf.int32)
    return features, label

ds_train = ds_train.map(preprocess).shuffle(100).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_validation = ds_vaidation.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(4,), dtype='float64'),
    keras.layers.Dense(12, activation='relu', dtype='float64'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(8, activation='relu', dtype='float64'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(6, activation='relu', dtype='float64'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(3, activation='softmax', dtype='float64')
])

# We use the sparse_categorical_crossentropy loss function when our dataset has labels that are integers, 
# and the categorical_crossentropy loss function when our dataset has one-hot encoded labels. 

# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/optimizers#classes

# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses#classes

# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/metrics#classes

# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#compile

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Evaluate the UNtrained model on the test dataset
test_loss, test_accuracy = model.evaluate(ds_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Train the model

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

history = model.fit(ds_train, epochs=EPOCHS, validation_data=ds_validation, callbacks=[early_stopping])

In [ ]:
# Evaluate the trained model on the test dataset
test_loss, test_accuracy = model.evaluate(ds_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get true labels and predictions from the test set
y_true = []
y_pred = []

for features, labels in ds_test:
    preds = model.predict(features)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=ds_info.features['label'].names)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Create a random tensor with the same shape as the input features (batch_size, 4)
# BATCH_SIZE
random_features = tf.random.uniform(shape=(1, 4), minval=4.0, maxval=8.0, dtype=tf.float64)

# Get model predictions for the random tensor
random_preds = model.predict(random_features)
predicted_classes = np.argmax(random_preds, axis=1)

print("Random input predictions:", predicted_classes)